# 네이버 검색 API - 뉴스 검색
* 뉴스 검색은 검색 API를 사용해 네이버 검색의 뉴스 검색 결과를 반환하는 RESTful API입니다.
* 뉴스 검색 결과를 XML 형식 또는 JSON 형식으로 반환합니다.
* API를 호출할 때는 검색어와 검색 조건을 쿼리 스트링(Query String) 형식의 데이터로 전달합니다.
* 
뉴스 검색은 검색 API를 사용하며, 검색 API의 하루 호출 한도**는 25,00**0회입니다.

In [1]:
import os
import sys
import urllib.request
import json
import pandas as pd
import re

# 셀레니움 임포트
from selenium import webdriver
import time

# chrome 브라우저 이용할 것
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

## 네이버 뉴스에서 '라인 사태'에 관한 뉴스 가져오기

In [5]:
client_id = "CGI0TWpzp1hxoq5hfXj8"
client_secret = "pd8aH97a5e"
query = urllib.parse.quote(input('검색 질의 : '))   #검색어 입력받음

idx = 0
display = 100    # 한 번에 보여줄 검색결과 개수
start = 1   # 검색 시작 인덱스
end = 1000   # 검색 끝 인덱스
sort = 'sim'    # 정확도순 내림차순 정렬 (date: 날짜순 내림차순 정렬)

news_df = pd.DataFrame(columns=['Title', 'Original Link', 'Link', 'Description', 'Publication Date'])

# 1 ~ 1000까지 100 간격으로 돌아감 (1 ~ 100, 101 ~ 200, ... , 901 ~ 1000)
for i in range(start, end, display):
    url = 'https://openapi.naver.com/v1/search/news.json?'
    url += ('query=' + query)
    url += ('&display=' + str(display))
    url += ('&start=' + str(i))
    url += ('&sort=' + sort)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']

        for item_index in range(0, len(items)):   # 각 기사의 정보가 리스트로 들어가 있음
            remove_tag = re.compile('<.*?>')   # 태그 제거

            title = re.sub(remove_tag, '', items[item_index]['title'])   # 리스트 안에는 딕셔너리 형태
            original_link = items[item_index]['originallink']
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            pub_date = items[item_index]['pubDate']
            news_df.loc[idx] = [title, original_link, link, description, pub_date]   # 데이터프레임에 삽입
            idx += 1

    else:
        print("Error Code: " + rescode)

# # 네이버 뉴스 외의 뉴스 링크도 포함
# news_df

# 네이버 뉴스만 포함 
news_df = news_df[news_df['Link'].str.contains('https://n.news.naver.com/mnews/article/')].reset_index(drop=True)

검색 질의 :  라인 사태


In [6]:
print(news_df.shape)
news_df.head(2)

(780, 5)


,Title,Original Link,Link,Description,Publication Date
0,[단독] 대통령실 &quot;日 보고서에 '지분 매각' 빠져…라인 사태 사실상 종료...,https://news.tvchosun.com/site/data/html_dir/2...,https://n.news.naver.com/mnews/article/448/000...,대통령실이 14일 '라인 사태'와 관련해 일본 정부의 행정지도 조치 보고서에 '지분...,"Tue, 14 May 2024 16:28:00 +0900"
1,"日언론, 라인야후 사태 한국 내 반일 감정 고조에 '촉각'",https://www.yna.co.kr/view/AKR2024051404100007...,https://n.news.naver.com/mnews/article/001/001...,다만 신문은 &quot;한국에서는 라인 사태와 관련해 '보안 문제와 자본은 관계가 ...,"Tue, 14 May 2024 09:56:00 +0900"


In [7]:
len(news_df['Title'].to_list())

780

## news_df의 'Link'에서 댓글 찾아서 댓글 링크 가져오기 + news_df에 컬럼 추가
* 'Link' 돌면서 댓글 가져오면 될 것 같은데..

In [2]:
# 원하는 내용이 포함된 DF에 댓글 링크 포함하는 함수
# '네이버 뉴스만 포함'시켜서 DF를 만들면 사용 가능

def add_comment_link(df):
    for i in range(df.shape[0]):
        # 변수에 링크 저장
        news_link = df.loc[i, 'Link']
        # / 로 news_link 나누기
        news_list = news_link.split('/')
        # 'comment' 삽입하기
        news_list.insert(5, 'comment')
        # 'news_list'를 다시 합치기
        comment_link = '/'.join(news_list)
        # DF에 댓글 링크 컬럼 추가하고 링크 넣기
        df.loc[i, 'CommentLink'] = comment_link
    # DF return
    return df

In [3]:
# 드래그 하기 함수
def drag_last(driver):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

In [4]:
# 하나의 기사의 댓글 가져와주는 함수
# 좋아요수, 싫어요수, 내용, 시간

def get_comments(link):
    # 웹드라이버 설정
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # 대기시간 : 10초
    driver.implicitly_wait(10)
    # 댓글을 찾고 싶은 페이지 열기
    driver.get(link)
        
    # '더보기'있는지 확인하기
    while True:
        drag_last(driver)
        time.sleep(1)
        
        # 더보기 있으면 클릭하기
        try:
            pagenation_element = driver.find_element(By.CSS_SELECTOR, 'div.u_cbox_paginate')
            pagenation_element.click()
        # 없으면 건너뛰기
        except:
            break
        
    # 크롬에서 댓글있는 박스 긁어오기
    comments_list = driver.find_elements(By.CSS_SELECTOR, '#cbox_module_wai_u_cbox_content_wrap_tabpanel > ul > li')
    
    # 담을 리스트 정하기
    text_list = []
    like_list = []
    unlike_list = []
    date_list = []
    
    # for문 이용해서 '작성시간', '댓글내용', '좋아요수', '싫어요수' 가져오기
    for comment in comments_list:
        try:
            
            text = comment.find_element(By.CSS_SELECTOR, 'div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_text_wrap > span.u_cbox_contents').text
            text_list.append(text)
            like = comment.find_element(By.CSS_SELECTOR, 'div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_tool > div > a.u_cbox_btn_recomm > em').text
            like_list.append(like)
            unlike = comment.find_element(By.CSS_SELECTOR, 'div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_tool > div > a.u_cbox_btn_unrecomm > em').text
            unlike_list.append(unlike)
            date = comment.find_element(By.CSS_SELECTOR, 'div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_info_base > span').text
            date_list.append(date)
        except:
            continue

    # print(text_list)
    # print(like_list)
    # print(unlike_list)
    # print(date_list)
    # 새로운 DF 만들기
    df = pd.DataFrame({'date' : date_list,
                           'comment' : text_list,
                           'like' : like_list,
                           'unlike': unlike_list})
    # DF return
    return df

In [8]:
# 댓글 링크 추가하기
news_df = add_comment_link(news_df)

In [9]:
# 추가되었나 확인
news_df[:10]

,Title,Original Link,Link,Description,Publication Date,CommentLink
0,[단독] 대통령실 &quot;日 보고서에 '지분 매각' 빠져…라인 사태 사실상 종료...,https://news.tvchosun.com/site/data/html_dir/2...,https://n.news.naver.com/mnews/article/448/000...,대통령실이 14일 '라인 사태'와 관련해 일본 정부의 행정지도 조치 보고서에 '지분...,"Tue, 14 May 2024 16:28:00 +0900",https://n.news.naver.com/mnews/article/comment...
1,"日언론, 라인야후 사태 한국 내 반일 감정 고조에 '촉각'",https://www.yna.co.kr/view/AKR2024051404100007...,https://n.news.naver.com/mnews/article/001/001...,다만 신문은 &quot;한국에서는 라인 사태와 관련해 '보안 문제와 자본은 관계가 ...,"Tue, 14 May 2024 09:56:00 +0900",https://n.news.naver.com/mnews/article/comment...
2,대통령실 &quot;라인야후 일 보고서에 '지분매각' 안 들어간다&quot;,https://www.newsis.com/view/?id=NISX20240514_0...,https://n.news.naver.com/mnews/article/003/001...,일본 정부의 지분 조정 요구로 촉발된 '라인야후 사태'가 한일 갈등으로 이어지자 우...,"Tue, 14 May 2024 16:27:00 +0900",https://n.news.naver.com/mnews/article/comment...
3,"與, '라인야후' 사태에 &quot;우리 기업 이익 최우선&quot;",http://news.tf.co.kr/read/ptoday/2098390.htm,https://n.news.naver.com/mnews/article/629/000...,"정점식 의장, 13~14일 과기정통부·네이버 의견 청취 &quot;네이버, 조속히 ...","Tue, 14 May 2024 14:40:00 +0900",https://n.news.naver.com/mnews/article/comment...
4,"라인야후 사태 속 한일 경제인회의...日 언론, 반일 감정 확산에 촉각",https://www.ytn.co.kr/_ln/0104_202405141701530719,https://n.news.naver.com/mnews/article/052/000...,"올해 56년째로 매년 정례적으로 열리지만, 이번에는 라인 야후 사태 속에서 개최되는...","Tue, 14 May 2024 17:01:00 +0900",https://n.news.naver.com/mnews/article/comment...
5,대통령실 “라인 부당조치땐 강력 대응”,https://www.seoul.co.kr/news/politics/presiden...,https://n.news.naver.com/mnews/article/081/000...,“우리 기업·국민의 이익 최우선” 지분 매각 등 네이버 입장은 존중 반일 프레임엔 ...,"Tue, 14 May 2024 05:03:00 +0900",https://n.news.naver.com/mnews/article/comment...
6,&quot;제2의 노노재팬 운동 일어날까&quot; '日 라인야후 사태' 불똥 어디로,http://www.fnnews.com/news/202405141040333174,https://n.news.naver.com/mnews/article/014/000...,다만 신문은 &quot;한국에서는 라인 사태와 관련해 '보안 문제와 자본은 관계가 ...,"Tue, 14 May 2024 10:45:00 +0900",https://n.news.naver.com/mnews/article/comment...
7,與 신지호 “‘라인야후’ 사태 장본인은 文… 2019년에 손정의 만나지 않았나...,http://www.segye.com/newsView/20240514503620?O...,https://n.news.naver.com/mnews/article/022/000...,뉴스1 자본 관계 재설정을 포함한 일본 정부의 행정지도로 촉발한 ‘라인야후 사태’ ...,"Tue, 14 May 2024 10:20:00 +0900",https://n.news.naver.com/mnews/article/comment...
8,'라인사태' 입장 밝힌 대통령실 &quot;반일 프레임 도움 안 돼&quot; [현...,https://imnews.imbc.com/news/2024/politics/art...,https://n.news.naver.com/mnews/article/214/000...,"━ 대통령실, '라인 사태' 관련 브리핑 2024년 5월 13일 [성태윤/대통령실 ...","Mon, 13 May 2024 18:17:00 +0900",https://n.news.naver.com/mnews/article/comment...
9,日언론 '화들짝'… 라인야후 사태 반일 감정에 '촉각',https://biz.sbs.co.kr/article_hub/20000171358?...,https://n.news.naver.com/mnews/article/374/000...,다만 신문은 &quot;한국에서는 라인 사태와 관련해 '보안 문제와 자본은 관계가 ...,"Tue, 14 May 2024 10:48:00 +0900",https://n.news.naver.com/mnews/article/comment...


# 댓글이 있는 페이지 크롤링하기
* 댓글 가져와서 news_df에 댓글 컬럼 만들어서 내용 넣기

In [10]:
# 브라우저 옵션 설정
chrome_options = Options()

# 창 닫기
chrome_options.add_argument('--headless')

# detach 옵션을 이용해 자동화
# 브라우저를 자동화한 후, browser window 창 유지
# chrome_options.add_experimental_option("detach", True)

# excludeSwitches : 불필요한 로깅 메세지 브라우저에서 제외
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 셀레니움에서 크롬 웹 드라이버를 자동으로 다운로드 및 설치
service = Service(executable_path=ChromeDriverManager().install())

# 함수 적용해서 댓글 관련 정보 가져오기
ex_comments_df = get_comments(news_df.iloc[0, 5])

In [12]:
ex_comments_df.sort_values('like', ascending=False)

,date,comment,like,unlike
59,2024.05.14. 17:02,라인사태를 왜 자꾸 정치적으로만 볼까. 라인은 문재인전대통령의 소개로 일본과 합작하...,9,2
50,2024.05.14. 17:13,매각도 안하는데 민주당은 호들갑 떨고 현정부와 국민들 속이는 형태에 대하여 국민앞에...,9,2
76,2024.05.14. 16:43,헛소리에 놀아나지 말고 끝까지 대응하시오. 일본에 속은게 어디 한두번이오.,7,0
71,2024.05.14. 16:48,하도 거짓말을 잘하는 윤정부와 일본넘들이니... 끝날때까지 끝난게 아닐 수도 있으니...,7,3
69,2024.05.14. 16:50,"일본정부가 말한 지분관계 재검토가 지분, 즉 주식 관계를 재검토 하라는 말이고 재검...",7,2
...,...,...,...,...
42,2024.05.14. 17:19,ㅋㅋㅋ 그치 ㅋㅋㅋ 오늘 썩렬 장모 출소 한단다 ㅋ 이젠 쪽팔린것도 민심도 안중에 ...,0,1
43,2024.05.14. 17:18,그래도 모르니 대비는 해야 한다고 생각함,0,0
44,2024.05.14. 17:18,개들이 판치는 나라에서. 개갔은 꼴보면서. 살아야하는. 국민들이 불쌍 하구나,0,0
45,2024.05.14. 17:18,일것이다 라는 확답없는 예상이라는 거네,0,0
